In [ ]:
! pip install selenium
! pip install undetected_chromedriver
! pip install python-dotenv

In [4]:
import os
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.support.select import Select
from dotenv import load_dotenv

In [7]:
# 데이터 엔지니어 
u0 = 'https://www.wanted.co.kr/salary/655' # 3,613
# u1 = 'https://www.wanted.co.kr/salary/655' # 3,773 -> url이 같아서 의미없음 -> 동적크롤링만이 답..?
# 야매로는 그냥 각 막대그래프의 height값으로 연봉 비교할 수 있긴함;;
# 도대체 구조가 어떻게 되어있는 것인가?
# -> 근데 또, html의 모든 값이 불러와지지 않음 -> ex)recharts클래스를 가진 path태그가 제대로 출력 안됨
    # 이유: 아래에서 lxml로 파싱했기 때문 -> html형식으로 파싱하면 됨
    # 아님. 정적크롤링으로는 안됨 https://www.inflearn.com/questions/1031893/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%9B%B9%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91-requests%EB%A5%BC-%ED%95%98%EB%A9%B4-%EC%9D%BC%EB%B6%80-%EC%86%8C%EC%8A%A4%EC%BD%94%EB%93%9C%EB%A7%8C-%EC%B6%9C%EB%A0%A5%EC%9D%B4-%EB%90%A9%EB%8B%88%EB%8B%A4
    # -> 따라서, 셀레니움으로 가져와야 함
    # +이유: JS로 실행되는 페이징을 크롤링 하기 위해서는 selenium 라이브러리를 사용하여 파이썬 코드에서 직접 화면의 JS를 실행시켜야 한다
a = requests.get(u0).text
a1 = bs(a, 'html') # a1 = bs(a, "lxml")
# print(a1.prettify())

a1.select('g.recharts') # [], 안나옴
# a1.select('h2.Info_jobCategorySalary__M4Ql6')[0].text # ' 3,613  만원'

[]

In [ ]:
driver = uc.Chrome()
driver.get('https://www.wanted.co.kr/salary/655')

soup= bs(driver.page_source, 'html.parser') # driver의 page_source를 가져오는 함수
# soup.prettify()

soup.select('')

In [64]:
# 아래처럼 하면 팝업창(사이트 권한 허용 관련)은 안뜸
# 대신, webdriver객체기 때문에 구글 로그인은 안됨..
# 구글 로그인 권한 푸는것하고 사이트 권한 푸는건 동시에 안되는듯..

# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--disable-notifications')  # Disable notifications 

# wd = webdriver.Chrome(options=chrome_options)
# wd.get('https://www.wanted.co.kr/salary/655')

In [11]:
# (1)권한 수락 팝업창떠서, 그걸 빠르게 허용을 눌러줘야 데이트가 제대로 뜸
    # ㄴㄴ 잘못 알고 있었음. 그냥, time.sleep()수치를 제대로 안 줘서 안 뜨는 것
    # 근데, 일반 webdriver.Chrome()객체로 하면 팝업창 수락 작동하는데,
    # undetected_chromedriver객체 쓰면 안됨

driver = uc.Chrome()
driver.get('https://www.wanted.co.kr/salary/518')
time.sleep(3) # 더 조정해보기

select1 = '#__next > div.Salary_Salary__OUYDA > div > div.SearchBar_SearchBar__8ma00 > div > div > div:nth-child(3) > div > select'
driver.find_element(By.CSS_SELECTOR, select1).click()
time.sleep(3)

select2 = '#__next > main > section > div > div > div > form > div.css-1qsnvsb > button.css-you85z > p'
driver.find_element(By.CSS_SELECTOR, select2).click()
time.sleep(3)


load_dotenv() # 이 함수를 한번 실행시켜야, .env파일에서 가져오기 가능
# google_id
driver.find_element(By.CSS_SELECTOR, '#identifierId').click()
driver.find_element(By.CSS_SELECTOR, '#identifierId').send_keys(os.getenv('GOOGLE_ID'))
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, '#identifierNext > div > button').click()
time.sleep(4)
# google_pw
driver.find_element(By.CSS_SELECTOR, '#password > div.aCsJod.oJeWuf > div > div.Xb9hP > input').click()
driver.find_element(By.CSS_SELECTOR, '#password > div.aCsJod.oJeWuf > div > div.Xb9hP > input').send_keys(os.getenv('GOOGLE_PW'))
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, '#passwordNext > div > button').click()
time.sleep(10)

# driver.find_element(By.CSS_SELECTOR, '#__next > div.Salary_Salary__OUYDA > div > div.SearchBar_SearchBar__8ma00 > div > div > div:nth-child(2) > div > select').click()
# time.sleep(3)
# select태그 안에 있는 것들 어떻게 선택? 일일이 select click시키면 안됨
# https://hoood.tistory.com/1973

# driver.find_element(By.CSS_SELECTOR, '#..').click()
# subCategory버튼을 click하는 코드 필요없음 -> 아래 select문으로도 충분함
select = Select(driver.find_element(By.NAME, 'subCategory'))
select.select_by_index(4) # 자바 개발자
time.sleep(3)
select.select_by_visible_text('데이터 엔지니어')
time.sleep(3)
select.select_by_value('677') # 안드로이드



soup = bs(driver.page_source, 'html.parser') # xlml이 아니라, html.parser로 해야 안 짤리고 전부 나옴
newbie = soup.select('h2.Info_jobCategorySalary__M4Ql6')[0].text # 신입연봉: ' 3,613  만원'
newbie = int(newbie[1:6].replace(',','')) # int: 3613

# 1. 신입,1~10 -> 신입의 height기준으로 나머지 비율구해서 연봉구하기 can;; - 개야매
len(soup.select('path.recharts-rectangle')) # 11
path_list = soup.select('path.recharts-rectangle')
height_list = [x['height'] for x in path_list]
height_list # 신입~10년차 막대의 height가 주어짐
salary_list = [int(float(y)*newbie/float(height_list[0])) for y in height_list]
print(salary_list) # [3613, 3772, 4138, 4557, 4984, 5041, 5461, 6054, 6426, 6513, 7698]

Python-dotenv could not parse statement starting at line 2


TypeError: 'NoneType' object is not iterable

In [15]:
a = os.getenv('GOOGLE_ID')
b = os.getenv('GOOGLE_PW')
print(a, b)

01030219040 None
